# Photosynthesis
::: {#exr-MM}
### Michaelis-Menten kinetics

Using `Cropbox`, create a function that implements the Michaelis--Mention equation as shown in @eq-MM that calculates the rate of biochemical reaction by an enzyme. Use the parameter values provided in @tbl-MM-parms. Plot the response of reaction rate ($V$) as a function of substrate concentration ranging from 0 to 1000. For this exercise we will ignore units but imagine that the substrate is used for carboxylation catalyzed by the enzyme Rubisco.
:::

::: {#tbl-MM-parms}
| Symbol | Value | Units | Description |
|:-|:-|:--|:--------|
| $V_{\mathrm{max}}$ | 100 | \[Product\] / time (*e.g.* $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$) | Potential rate of biochemical reaction for product synthesis (*i.e.* Rubisco capacity) |
| $K_m$ | 400 | \[Substrate\] (*e.g.* $\mathrm{M}$, $\mathrm{ppm}$, $\mathrm{\mu mol}\ \mathrm{mol^{-1}}$) | Michaelis-Menten constant where $V = \frac{1}{2} V_{\mathrm{max}}$ |

Michaelis-Menten parameters
:::

In [ ]:
#| output: false
using Cropbox

@system MichaelisMenten(Controller) begin
    V(S, Vmax, Km): reaction_rate => begin
        Vmax * S / (Km + S)
    end ~ track(u"μmol/m^2/s")

    Vmax: maximum_reaction_rate     => 100 ~ preserve(parameter, u"μmol/m^2/s")
    Km:   michaelis_menten_constant => 400 ~ preserve(parameter, u"μmol/mol")

    S:    substrate                 => 0   ~ preserve(parameter, u"μmol/mol")
end

In [ ]:
instance(MichaelisMenten)

::: {#tip-config-sims .callout-tip}
### Steady-state simulations using `@config`

By default, `simulate()` runs a single simulation for a given configuration set from `config` option. If we want to run multiple simulations, a list of multiple configurations should be provided in `configs` option (note trailing `s`). Cropbox provides a helper macro `@config` to easily generate such a list of multiple configurations useful for some common use cases such as adjusting one parameter over a range of values. Here we want `S` value ranging from 0 to 1000 by 1 (`0:1:1000` or `0:1000` as the step size is 1 by default) which is declared as a Julia built-in range type and later expanded to a list by `!` operator used in conjunction with the `@config` macro. By the way, the name of system `:0` in the configuration is a shorthand for the name of whatever top-level system currently used for simulation. In this example, it is going to be `MichaelisMenten`.

In [ ]:
mmc = @config !(:0 => :S => 0:1000);

In [ ]:
r = simulate(MichaelisMenten; configs = mmc);

Note that `time` doesn't advance here indicating that this is a `steady-state` simulation

In [ ]:
visualize(r, :S, :V; kind = :line);

We can use another form of `visualize()` for automatically running `simulate()` inside. A good thing about this approach is that we don't have to deal with custom expansion of configurations using `@config` in favor of convenient options like `xstep`. `xstep` indicates the range of a certain parameter value that needs to be simulated individually to make a series of consecutive output. Here we change the value of `S` from 0 to 1000 with an increment of 10 by simply providing a snippet of range value (`:0 => :S => 0:10:1000`) representing 101 times of simulation. Common plotting options like `kind` are also available.

In [ ]:
visualize(MichaelisMenten, :S, :V; 
    xstep = :0 => :S => 0:10:1000,
    kind = :line,
);

Create a configuration to change `Km` and `Vmax`, and observe how `V` changes with different`Km` and `Vmax` values over a wide range of `S` levels

In [ ]:
c1 = @config(
    :MichaelisMenten => (
        Km = 200,
        Vmax = 50,
        ),
    )

Run the simulation with new configuation `c1` over a range of `S` specified in `mmc`.

In [ ]:
r = simulate(MichaelisMenten; configs = mmc, config = c1);

We can simply `visualize` this using `xstep` argument with new configuration `c1`.

In [ ]:
visualize(MichaelisMenten, :S, :V; 
    config = c1,
    xstep = :0 => :S => 0:10:2000,
    kind = :line,
)

:::

### Quadratic equation

A **quadratic equation** refers to an equation that can be rearranged into the form:

$$
    ax^2 + bx + c = 0
$$ {#eq-quadratic}

A quadratic equation can be factored to get the two roots but more general solutions (i.e., the roots) can be found using the **quadratic formula**.

$$
    x=\frac{-b\pm\sqrt{b^2 - 4ac}}{2a}
$$ {#eq-quad-formula}

Rearranging variables (e.g., assimilation, light) and associated parameters (e.g., quantum efficiency) into a quadratic equation is a useful technique in modeling because one can obtain a root that provides biological or physical insights in regards to the response of this variable to rate limiting processes or factors using the quadratic formula. Although less commonly used, the same can apply to the cubic equation and cubic formula (see <https://math.vanderbilt.edu/schectex/courses/cubic/>).

## Leaf photosynthesis

### Response to light and CO~2~

One of the most well-known characteristics about photosynthesis is that it increases and saturates to an asymptote at very high levels of light and , respectively. This photosynthetic response to light and was recognized early. A quantitative study by [@Blackman1905] that goes back to early 1900 identified light and \[\] as two key limiting factors and provides a simple but robust model often referred to as "the Blackman response".

![The Blackman response. Theoretical photosynthetic CO~2~ response at different light intensities.](./figs/blackman_curve.png){#fig-blackman width="90%"}

Deriving from it, there are many ways to depict the photosynthetic responses of a leaf mathematically [See @Archon2015; @Thornley2000]. We will review a few of them here that are commonly used for depicting light response curves. These models are relatively simple and often used as submodels to describe photosynthetic light responses within crop or forest growth models. @Thornley2000 [Ch. 9] provides mathematical derivations of these models with physiological underpinnings. A simple but versatile model that reflects the light response under a given in the air ($C_a$) can be shown as:

$$
    \theta A_g^2 - (\alpha I + A_{\mathrm{max}}) A_g + \alpha I A_{\mathrm{max}} = 0
$$ {#eq-quadratic-Ag}

$A_g$ represents the gross assimilation rate of leaf. $\alpha$ is the initial slope of the curve representing the photochemical efficiency (aka, quantum efficiency). $A_{\mathrm{max}}$ is the asymptote that represents the maximal rate of photosynthesis at saturating light reflecting the photosynthetic capacity of the leaf. $\theta$ determines curvature of the transition between light limited and light saturated photosynthesis. Physiologically $\theta = r_d/(r_d+r_x) = g_x/(g_x+g_d)$. $r_d$ and $r_x$ are diffusion and carboxylation resistances of (s m^−1^), respectively and $g_d$ and $g_x$ are reciprocals representing the corresponding conductance terms.

![Sensitivity of non-rectangular hyperbola with respect to $\theta$](./figs/NRH.png){#fig-NRH width="80%"}

When $\theta$ = 1, @eq-quadratic-Ag can be factorized to give a limiting response of two straight lines intersecting at $I = A_{\mathrm{max}}/\alpha$. This simple model represents the Blackman response @fig-blackman:

$$
    A_n = \min(\alpha I, A_{\mathrm{max}}) - R_d
$$ {#eq-blackman}

Where $A_n$ stands for the net assimilation rate of a leaf and $R_d$ for the rate of mitochondrial respiration for the same leaf.

When $\theta$ = 0, @eq-quadratic-Ag reduces to a rectangular hyperbola:

$$
A_n = \frac{\alpha I A_{\mathrm{max}}}{\alpha I + A_{\mathrm{max}}} - R_d
$$ {#eq-rectangular}

A more generalized equation, non-rectangular hyperbola, is obtained from taking the lower root of the quadratic equation @eq-quadratic-Ag for 0 \< $\theta$ \< 1:

$$
A_n = \frac{\alpha I + A_{\mathrm{max}} - \sqrt{(\alpha I + A_{\mathrm{max}})^2 - 4\theta\alpha I A_{\mathrm{max}}}}{2\theta} - R_d
$$ {#eq-NRH}

The non-rectangular hyperbola @eq-NRH is a simple but versatile expression for the light response of leaf photosynthesis with the parameters ($A_{\mathrm{max}}, \alpha, \theta$) that have physiological meaning and gives an excellent fit to experimental data of leaf photosynthesis [@Thornley2000].

::: {#nte-other-factors .callout-note}
### Environmental effects on photosynthesis
In addition to light, leaf photosynthesis responds to other environmental factors. What are those environmental factors that are important for photosynthesis but not addressed by the models covered in this section so far?

How would you incorporate these factors into the models we looked at in this section?
:::

::: {#exr-leaf-A}
### Leaf photosynthesis models

Using `Cropbox`, let's create a function to predict leaf net photosynthesis ($A_n$) as a function of irradiance ($I$) that takes parameters of photochemical efficiency ($\alpha$; a.k.a., quantum efficiency), the maximum photosynthetic rate at saturating light ($A_{\mathrm{max}}$), and dark respiration rate ($R_d$) based on typical parameter values provided in @tbl-Aleaf-params. 

1.  Based on rectangular hyperbola @eq-rectangular. 

2.  Based on non-rectangular hyperbola @eq-NRH for ${0 < \theta \le 1}$)

3.  Using the parameter values in @tbl-Aleaf-params, plot the light response curves of both functions from 0 to 2000 μmol m^2^ s^−−1^ of photosynthetic photon flux density (PFD).

4.  Evaluate the sensitivity of non-rectangular hyperbola to $\theta$ graphically and discuss.\

::: {#tbl-Aleaf-params}
| Symbol | Value | Units | Description |
|:-|:-|:--|:--------|
| $\alpha$ | 0.05 | $$\mathrm{\mu mol_{CO_2}}\ \mathrm{\mu mol_{photon}^{-1}}$$ | Apparent photochemical efficiency (*a.k.a.* quantum yield) |
| $A_{\mathrm{max}}$ | 25.0 | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Light saturated $A_g$ at ambient $\mathrm{[CO_2]}$ and standard temperature |
| $R_d$ | 1.0 | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Leaf mitochondrial respiration rate at 25 $\mathrm{^{\circ}C}$ |
| $\theta$ | 0.7 | \- | Curvature determining transition between light-limited and saturated photosynthesis |
| $I$ | \- | $\mathrm{\mu mol_{photon}}\ \mathrm{m^{-2}}$ | Incident photosynthetic photon flux density (PFD) on the leaf surface |
| $A_g$ | \- | $$\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Leaf gross $\mathrm{CO_2}$ assimilation rate |
| $A_n$ | \- | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Leaf net $\mathrm{CO_2}$ assimilation rate |

Variables and parameters of leaf photosynthesis models
:::

### Template `Photosynthesis` system { .unnumbered}

We first define a mix-in system named `Photosynthesis` which provides a shared template of photosynthesis model which other models will be all based on.

We have three common parameters and two variables representing photosynthesis rate. $A_n$ is the net photosynthesis derived from gross photosynthesis ($A_g$) and respiration ($R_d$). As we decided to make $R_d$ a constant parameter, subsequent models would only need to provide a custom definition for $A_g$. Although `Ag` is not defined here, it is still a dependency of `An` and thus we need to make a placeholder for it by creating a `hold` variable. For declaring `hold`, only a name and an optinal alias of the variable are needed. Any variables with `hold` state should be properly defined later in another system which embeds `Photosynthesis` as a mix-in.

In [ ]:
@system Photosynthesis begin
    α:    photochemical_efficiency    => 0.05 ~ preserve(parameter)
    Amax: maximum_photosynthetic_rate => 25   ~ preserve(parameter, u"μmol/m^2/s")
    Rd:   dark_respiration            => 1.0  ~ preserve(parameter, u"μmol/m^2/s")

    Ag: gross_photosynthesis ~ hold

    An(Ag, Rd): net_photosynthesis => Ag - Rd ~ track(u"μmol/m^2/s")
end;

Here is our first model based on rectangular hyperbola named `PhotosynthesisRH` which consists of two mix-ins, `Photosynthesis` and `Controller`. As you remember, `Controller` is a mix-in that makes a top-level system which can be instantiated by `instance()` or `simulate()`. Note that variables from other mix-in that are used by the system should be declared as `hold` kind of variable to ensure correct bindings between variables. Note that `PhotosynthesisRH` now has a concrete definition of `Ag` with an introduction of additional parameter `I`.

### Rectangular hyperbola { .unnumbered}

In [ ]:
@system PhotosynthesisRH(Photosynthesis, Controller) begin
    I: irradiance => 0 ~ preserve(parameter, u"μmol/m^2/s")

    Ag(I, α, Amax): gross_photosynthesis => begin
        α*I * Amax / (α*I + Amax)
    end ~ track(u"μmol/m^2/s")
end;

We use `visualize()` with a range of `I` value set up for `xstep` in the same way as we saw earlier.

In [ ]:
visualize(PhotosynthesisRH, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
);

### Non-rectangular hyperbola { .unnumbered}

Another variant of the model using non-rectangular hyperbola looks similar to the rectangular model we built earlier, except the addition of new parameter `θ` controlling curvature of transition.

In [ ]:
@system PhotosynthesisNRH(Photosynthesis, Controller) begin
    θ: transition_curvature => 0.7 ~ preserve(parameter)
    I: irradiance           => 0   ~ preserve(parameter, u"μmol/m^2/s")

    Ag(θ, I, α, Amax): gross_photosynthesis => begin
        (α*I + Amax - √((α*I + Amax)^2 - 4θ*α*I*Amax)) / 2θ
    end ~ track(u"μmol/m^2/s")
end;

In [ ]:
visualize(PhotosynthesisNRH, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
);

Instead of solving the quadratic equation for $A_g$ manually as we did above, we can use `solve` variable to let the framework automatically figure it out. In such case, we can use Eqn\[5.9\] almost as is. When multiple solutions exist, we may need to provide an extra information to guide a solution. Here we specify a sensible range of $A_g$ from 0 to $A_{\mathrm{max}}$ with `lower` and `upper` tags.

In [ ]:
@system PhotosynthesisNRH2(Photosynthesis, Controller) begin
    θ: transition_curvature => 0.7 ~ preserve(parameter)
    I: irradiance           => 0   ~ preserve(parameter, u"μmol/m^2/s")

    Ag(Ag, θ, I, α, Amax): gross_photosynthesis => begin
        θ*Ag^2 - (α*I + Amax)*Ag + α*I*Amax
    end ~ solve(lower = 0, upper = Amax, u"μmol/m^2/s")
end;

In [ ]:
visualize(PhotosynthesisNRH2, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
);

### Sensitivity of θ {.unnumbered}

So far we've only made plots with a single line composed of a series of points from multiple simulations triggered by `xstep` option. For drawing multiple lines of simulations for non-rectangular hyperbola, we use `group` option to control another parameter value in the system. For testing sensitivity of `θ`, we want to run simulations with different values of `θ` from 0.2 to 1.0 by increment of 0.2. For convenience of the reader, we use a parameter list in reverse order (`[1, 0.99, 0.9, 0.7, 0.4]`) to show the labels from top to bottom. Additionally, we append a plot for the rectangular hyperbola obtained when `θ` is 0.

In [ ]:
let x = :I, y = :An,
    xstep = :0 => :I => 0:10:1500,
    group = :0 => :θ => [1, 0.99, 0.9, 0.7, 0.4],
    kind = :line
    p = visualize(PhotosynthesisNRH, x, y; group, xstep, kind)
    visualize!(p, PhotosynthesisRH, x, y; xstep, kind, name = "0")
end

:::

:::{#nte-acock-model .callout-note}
### Two variable rectangular hyperbola
The rectangular hyperbola model can be extended to respond two variables simultaneously. Here is an example of this model in response to light ($I$) and atmospheric CO~2~ ($C_a$)(@Acock1978. This model is based on the assumption that the asymptotic response to light or CO~2~ is limited by the other variable. 

$$
A_n = \frac{\alpha I \tau C_a}{\alpha I + \tau C_a} - R_d
$$ {#eq-acock}
:::

### The FvCB model

In 1980, Farquhar and colleagues published a biochemical model for C3 plants that attempted to incorporate and simplify the expanding understanding of mechanisms behind photosynthesis (The FvCB model; [@Farquhar1980]). The FvCB model was further modified by Harley and Sharkey ([-@Harley1991]), and was later expanded for C4 photosynthesis by von Caemmerer and Furbank ([-@vonCammerer1999]). Despite being published decades ago, the FvCB model is still one of the most widely used approaches for modeling photosynthetic response to environmental factors and the backbone of many ecosystem and earth system models [@Lochocki2025; @Walker2021]. We will focus on this approach starting with a brief overview the FvCB model. See [@vonCaemmerer2000] and [@vonCaemmerer2009] for more comprehensive reviews.

Borrowing from the econometrics, the FvCB model captures the biochemical **'demand'** for assimilation at the chloroplast that is scalable to a leaf. The leaf net assimilation rate ($A$) is determined by describing the rate of three key processes within the Calvin cycle: 1) the rate of carboxylation, which is catalyzed by Rubisco ($A_c$), 2) the rate of RuBP regeneration, which is controlled by the electron transport rate within the light-dependent reaction of photosynthesis ($A_j$), and 3) the rate of synthesizing photosynthetic end products ($A_p$), mainly, converting triose phosphates into sucrose and starch, also known as triose phosphate utilization (TPU) @fig-rate-limiting-steps. Since the three processes (i.e. $A_c, A_j, A_p$) are continuous components within the Calvin cycle, the most rate limiting process eventually determines the net photosynthetic rate @eq-A:

$$
    A = \min\{A_c, A_j, A_p\}
$${#eq-A}

The rate of each process responds to enzymatic properties as well as environmental variables such as temperature, light levels, and and concentrations. This model as represented here comes with an important caveat that it operates with an assumption that the CO~2~ concentration at the intercellular air spaces (C~i~) is known and that C~i~ is identical as the CO~2~ concentration at the site of carboxylation (C~c~) in the chloroplast. For unstressed C3 leaves, the ratio between C~i~ and C~a~ (C~i~/C~a~) has been found to hover around 0.7 [@Ehleringer1995; @Tan2017; @Peters2018]. However, this ratio will change as plants experience drought and other stresses [@Brodribb1996] Coupling processes between biochemical demand for CO~2~ and biophysical supply of CO~2~ through stomata is an important step in utilizing the FvCB model for realistic predictions [@Kim2003]. For our purpose to illustrate the model, we will assume that `C~i~` values are known.

Here, we briefly describe how the FvCB model individually describes $A_c$, $A_j$, and $A_p$. First, $A_c$ represents the photosynthetic rate as assimilation rate limited by the carboxylation capacity of Rubisco, and is calculated based on Rubisco kinetics @eq-Ac:

$$
A_c = \frac{(C - \Gamma^*) V_{\mathrm{cmax}} }{C + K_c (1+ O /K_o)} - R_d
$${#eq-Ac}

$V_\mathrm{cmax}$ describes the maximal rate of carboxylation for Rubisco, and $K_c$ and $K_o$ are the Michaelis-Menten constants for Rubisco reacting with and , respectively. These parameters describe the enzymatic properties of Rubisco, which are temperature-dependent. $C$ and $O$ denote the and partial pressure at the chloroplast, and $\Gamma^*$ is the compensation point, defined as the concentration level at which carbon uptake through photosynthesis is balanced by released through photorespiration. $R_d$ is the respiratory release of excluding that from photorespiration.

The second term, $A_j$ describes the assimilation rate limited by RuBP regeneration, and is calculated based on the electron transport rate within the light reaction of photosynthesis that generates ATP and NADPH to fuel RuBP regeneration @eq-Aj:

$$
A_j = \frac{(C - \Gamma^*) J}{4C + 8\Gamma^*} - R_d
$${#eq-Aj}

$J$ describes the electron transport rate that is a function of light absorbed in photosynthetically active radiation, typically modeled as a saturating response curve using non-rectangular hyperbola [@vonCaemmerer2009; @Kim2020; @Kim2003].

Finally, $A_p$ represents the rate of assimilation limited by triose phosphate utilization (TPU), and is calculated based on the rate at which the photosynthetic end products are being utilized and transported out of the chloroplast @eq-Ap:

$$
A_p = 3 TPU - R_d
$${#eq-Ap}

![Simplified schematic of the rate limiting processes in C3 photosynthesis adapted from [@Sharkey2007]](./figs/rate_limiting_steps.png){#fig-rate-limiting-steps fig-align="center"}

Note that all biochemical parameters depend on temperature. The Arrhenius equation is a commonly used approach for modeling temperature dependence of photosynthetic parameters. See [@Kim2003; @Yun2020; @Medlyn2002] for the form and implementation of this function.

::: {#nte-ci-ratio .callout-note appearance="default"}
### C~i~/C~a~ ratio
Think and discuss whether, how, and why the C~i~/C~a~ ratio will change in response to drought stress in C3 plants.
:::

::: {#exr-FvCB-model}
### Behavior of the FvCB model

Implmenting the FvCB model line by line is a time-consuming, complex task. We will import a published Julia package: [`LeafGasExchange.jl`](https://github.com/cropbox/LeafGasExchange.jl) to illustrate the model behavior [@Yun2020]. We will revisit this later when we delve into coupled leaf gas-exchange modeling.

1.  Evaluate the model behavior with defalut parameter values for responses to light, CO~2~, and temperature
2.  Replace the parameter values from Homework
:::

In [ ]:
using LeafGasExchange: C3

In [ ]:
@system C3leaf(C3, Controller) begin
     T:  leaf_temperature  ~ preserve(parameter, u"°C")
     Ci: intercellular_co2  ~ preserve(parameter, u"μbar")
     I2: effective_irradiance ~ preserve(parameter, u"μmol/m^2/s")
end

In [ ]:
cleaf = @config (
# estimates of Vcmax, Jmax, and Rd at 25C for a rose leaf (Kim and Lieth, 2003)
# replace these values with yours from hw 1
    #  :C3 => (
    #       Vcm25 = 102.4, 
    #       Jm25 = 162.0, 
    #       Rd25 = 1.260,
    #  ), 
# provide environmental data needed to run the FvCB model    
    :0 => (
        I2 = 1500,
        Ci = 300,
        T = 25,
    )
);

``` julia
Cropbox.dependency(C3leaf); Cropbox.hierarchy(C3leaf)
```

In [ ]:
visualize(C3leaf, :Ci, [:A_net, :Ac, :Aj, :Ap];
    config = cleaf,
    xstep = :0 => :Ci => 0:1000,
    ylim = (0,50),
    kind = :line,
)

::: {#tip-coupled-model-demo .callout-tip}
### Coupled model example
We can run a coupled model for rose leaves and evaluate the model behavior for a quick comparison.

``` julia
using LeafGasExchange: ModelC3BB as C3BB

c_rose = @config (
# coupled model parameter estimates for a rose leaf (Kim and Lieth, 2003)    
    :C3 => (
        Vcm25 = 102.4, Jm25 = 162.0, Tp25 = 11.55, Rd25 = 1.260,
        Kc25 = 404, Ko25 = 248, Eac = 59.4, Eao = 36.0, Ear = 66.4, EaVc = 45.5,
        Eaj = 43.3, Hj = 219.4, Sj = 704.2, Γ25 = 36.9,
    ),
    :StomataBallBerry => (g0 = 0.096, g1 = 10.055), 
    :Weather => (
        PFD = 1500,
        CO2 = 400,
        RH = 60,
        T_air = 25,
        wind = 2.0,
    )
)
```

``` julia
visualize(C3BB, :CO2, [:A_net, :Ac, :Aj, :Ap];
    config = c_rose,
    xstep = :Weather => :CO2 => 50:10:1500,
    ylim = (0,50),
    kind = :line,
)
```
:::

::: {#exr-fvcb-responses}
### FvCB mode: Light and temperature responses. 
Apply the FvCB model to generate light (PFD) and temperature response curves for a C3 leaf of your choice. Identify the regions limited by each of the three steps (i.e., $A_c$, $A_j$, or $A_p$). Assume that all other conditions are favorable for photosynthesis in ambient conditions except the variable of interest.
:::

## Gaussian integration

In most process-based plant growth simulation models, the biomass accumulation is calculated using a numerical integration method. Numerical integration becomes necessary because the analytical solutions are often unachievable or unavailable. However, depending on the method and integration steps used for numerical integration, the numerically integrated values may have considerable errors. Decreasing the integration step ($\Delta t$) will reduce the errors but increase the execution time and computational load. It is good practice to evaluate the error due to numerical integration. Most widely accepted numerical integration method is Eulerian (rectangular) method which is also the simplest [@Thornley2000; @Thornley2007]. Other commonly used methods include trapezoidal and Runge--Kutta methods @fig-num-int. When the function that determines the rate is known, Gaussian integration method can be an effective alternative for its efficiency and accuracy [@Goudriaan1986]. We will work on some examples using this method from [@G&vL-1994]. We will revisit this subject later to apply Gaussian integration for modeling canopy photosynthesis. A good understanding about pros and cons of numerical integration methods is important for dynamic modeling.

![Numerical integration methods [adapted from @Goudriaan1986].](./figs/num_int.png){#fig-num-int width="80%"}

::: {#nte-sinx .callout-note}
### Gaussian integration example
Work on solving @eq-sinx using analytical, Gaussian two-point, and three-point integration methods. See examples in Appendix 2 of [@G&vL-1994].

$$
\int_0^{\pi} \sin{x} \, dx
$$ {#eq-sinx}

1.  Get an analytical solution for @eq-sinx

2.  Create a function to get numerical solutions using Gaussian two-point and three-point integration methods.

#### Analytical solution {.unnumbered}

In [ ]:
f′(x) = sin(x)
f(x) = -cos(x)
f(π) - f(0);

#### Gaussian two-point integration method {.unnumbered}

As the two points (-1/√3, +1/√3) and weights (1 for both) are defined in a normalized interval \[-1, 1\], we need to shift and scale them to the interval that our problem lies on, \[0, π\]. This process is shown step by step below.

##### \[-1, 1\] {.unnumbered}

In [ ]:
(1 * f′(-1/√3) + 1 * f′(1/√3)) * 2/2;

##### \[0, 2\] {.unnumbered}

In [ ]:
(1 * f′(-1/√3 + 1) + 1 * f′(1/√3 + 1)) * 2/2;

##### \[0, 1\] {.unnumbered}

In [ ]:
(1 * f′((-1/√3 + 1) / 2) + 1 * f′((1/√3 + 1) / 2)) * 1/2;

##### \[0, π\] {.unnumbered}

In [ ]:
(1 * f′((-1/√3 + 1) / 2 * π) + 1 * f′((1/√3 + 1) / 2 * π)) * π/2;

#### Gaussian three-point integration method {.unnumbered}

##### \[0, π\] {.unnumbered}

In [ ]:
(5/9 * f′((-√(3/5) + 1) / 2 * π) + 8/9 * f′((0 + 1)/2 * π) + 5/9 * f′((√(3/5) + 1) / 2 * π)) * π/2;

Calculation of Gaussian quadrature can be more easily done via a dedicated package called [QuadGK.jl](https://github.com/JuliaMath/QuadGK.jl) which is already imported inside Cropbox.

In [ ]:
X, W = Cropbox.QuadGK.gauss(2, 0, pi)
sum(W .* f′.(X));

In [ ]:
X, W = Cropbox.QuadGK.gauss(3, 0, pi)
sum(W .* f′.(X));

:::

## Canopy photosynthesis

### Brief history

Some of the earliest attempts to model canopy photosynthesis focused on describing the light interception and distribution within the canopy. Leaf-level photosynthetic rates were modeled as a function of the intercepted light level at different canopy layers, commonly through a rectangular hyperbola function [@Duncan1967; @Monsi2005]. Subsequently, based on field observations that biomass accumulation at the canopy level is proportional to intercepted radiation, [@Monteith1977] presented how dry matter production (as a measure of plant productivity) of various crops was linearly correlated with the intercepted radiation under ideal growing conditions, leading to the theory that carbon gain in crops through photosynthesis can be estimated based on the amount of sunlight intercepted during the growing season, and the efficiency with which it is converted into biomass. This concept is commonly known as radiation use efficiency (RUE), and has been adopted into many crop simulation models due to its elegance and simplicity [@White2011]. But this approach is not the focus of this lesson.

### Radiation Use Efficiency

Early on it has been recognized that the crop productivity is linearly related to the amount of radiation absorbed by a crop canopy, and this relationship is well conserved across different locations. This concept of radiation use efficiency proposed by @Monteith1977 is simple and elegant.

![The relationship between biomass accumulation and photosynthetically active radiation (PAR) absorbed by crop canopy in maize. The slope of this relationship represents the radiation use efficiency. Adopted from [@Lindquist2005]](./figs/maize_rue.png){#fig-maize-rue width="70%"}

Based on the thermodynamic efficiency of a crop canopy to convert the light energy (MJ m^−2^ in short-wave solar radiation) into the chemical energy to produce carbohydrates forming crop biomass (g m^−2^ in dry matter), it has been thought that RUE remains conservative among different crops and environmental conditions. Later works provided theoretical underpinnings of this approach [@Loomis1999]. The RUE approach can be summarized as:

$$
\Delta W = [\epsilon\cdot I_0\cdot f_{\mathrm{solar}}]\Delta t
$${#eq-rue}

Where $W$ is biomass accumulated per unit area (g m^−2^), $I_0$ represents the irradiance in short-wave solar radiation (PAR and NIR wavebands) and is usually available from weather station data in W m^−2^ or in MJ m^2^ d^−−1^. While RUE values are most commonly expressed for the solar radiation including both PAR and NIR wavebands, you may encounter that some RUE values are expressed based only on PAR in different studies and models. Therefore, paying close attention to the wavebands and unit is needed. $\epsilon$ is a crop specific coefficient for its radiation use efficiency (g MJ^−1^), $f_{\mathrm{solar}}$ is the fraction of the solar radiation absorbed by the canopy. $\Delta t$ is the time step, usually in d. $f_{\mathrm{solar}}$ can be approximated by:

$$
f_{\mathrm{solar}} = 1 - e^{-kL}
$${#eq-fsolar}

Where $k$ is extinction coefficient that describes the rate of light attenuation inside a canopy and $L$ is the leaf area index of the canopy. Typical values for the RUE coefficient ($\epsilon$) are 1.5 and 2.3 g MJ^−1^ for C3 and C4 crops, respectively [@Loomis1999]. These values are based on solar radiation (i.e., PAR + NIR). PAR based values will be usually higher [e.g. @Lindquist2005], about twice of these values because PAR consistes of approximately 45 to 50 % of solar radiation. A typical value for $k$ may range between 0.5 and 0.8 with 0.5 representing a canopy with spherical leaf angle distribution.

$$
    W = \sum_{t=1}^{d_{last}}[\epsilon\cdot I_0\cdot f_{\mathrm{solar}}]
$${#eq-rue-sum}

@eq-rue can be re-written as @eq-rue-sum to give the final biomass at the end of a growing season ($d_{last}$) to account for net total biomass accumulated over the entire growing season. For an ecosystem (e.g., forest, desert), this quantity will be equivalent to the net primary productivity (NPP) for a given year. In both cases, the leaf area index (LAI) needs to be known as a parameter or an input variable, which is often estimated using remote sensing data. Once final biomass accumulated over the season is known, the harvested crop yield ($Y_h$) can be obtained if harvest index ($h$) of the crop is known. The $h$ for a highly domesticated and bred grain crop (e.g., corn, rice, wheat) may be up to 50% or greater.

$$
Y_h = W\cdot h
$${#eq-yield-hi}

While the RUE approach for modeling crop or ecosystem productivity is elegantly simple and effective, the assumed linear relationship between the radiation intercepted by the canopy and biomass production may not be conserved in all environments.

### Instantaneous canopy photosynthesis

So far we've looked at ways to model leaf-level photosynthesis in response to light and potentially other environmental factors. But a whole-plant is a collection of leaves, and many plants form a closed or partially closed canopy together. Most crop canopies tend to be structurally homogeneous while forest canopies are structurally more complex and heterogeneous. The process of scaling up photosynthesis from a leaf to the canopy is critical for an accurate estimation of crop productivity and of the primary productivity of an ecosystem.

#### Integrative approaches

There are various modeling methods to estimate the canopy productivity, one being the RUE approach we just looked at above. The RUE approach is equivalent to what's referred as **big-leaf model** because it assumes that the canopy behaves like a big single leaf without distinguishing functional traits (e.g., photosynthetic capacity, N content, angle, shape, thickness) of different leaves or components in the canopy. The big-leaf models are deemed overly simplistic with a tendency to overestimate the productivity of dense canopy (i.e., high LAI). Another approach is to divide the canopy into many layers and therefore known as **multi-layer model**. In each layer, leaves are assumed to share similar functional traits with random distribution. This one-dimensional multi-layer approach can be further extended to two-dimensional grids or three-dimensional voxels under the same principles if the traits and conditions are known with an increase in dimension. The multi-layer approach is thought to be more accurate than the big-leaf approach but computationally demanding and has extensive parameter requirements. A simpler but equally effective alternative of the multi-layer approach is known as **sun-shade leaf model** [@dePury1997]. This modeling approach divides a canopy into sunlit and shaded fractions. Leaves in each leaf class shares the same functional traits (e.g., N content, photosynthetic capacity, morphology, light and other microclimate). Key distinctions between sunlit and shaded leaf classes are found in their light environment and photosynthetic capacity. Sunlit leaves receive both direct and diffuse light while shaded leaves see only diffuse light. As this difference in their light environment persists as the leaves grow, they then acclimate to develop different photosynthetic capacity which correlates to the N content. The N distribution in the canopy often closely follows the light distribution. The sun-shade leaf approach is simple and computationally efficient. Yet it performs as accurately as the multi-layer models, and has become a widely accepted choice of Earth System Models and newer crop models. [@dePury1997] provides a comprehensive overview of these modeling approaches with sufficient specifics for reproducing the models.

#### Functional--Structural Plant Models

A different approach worth noting here is the **Functional--Structural Plant Models (FSPM)**. These models focus on realistic representation of 3D plant structure of an individual plant. The structural component is coupled with physiological functions that drive the growth at the organ or sub-organ level. This approach differs from the previous approaches we discussed so far in that FSPM simulate individual plants separately while other scaling methods assume that plants in the canopy share identical traits within a modeling unit (e.g., canopy, layer, grid, voxel). Thus, FSPM are suitable for addressing questions related to plant-plant interactions, plant morphology and microclimate (e.g., light) interactions in the canopy. We are not going to cover FSPM in detail here but see the reviews by [@Evers2020] and [@Vos2010] if you are interested in learning more about FSPM. [`VirtualPlantLab.jl`](https://github.com/VirtualPlantLab) is a Julia package to implement FSPMs in Julia.  

#### An analytical integration of canopy photosynthesis for monocultures

To obtain canopy photosynthesis from an integration of leaf photosynthesis, the attenuation of light inside the canopy needs to be accounted for. The light interception by the leaves is described by Beer's law as a function of cumulative leaf area index ($l$). In its simplest form, this is:

$$
\begin{aligned}
    I_h(l) &= I_0 e^{-k l} \\
    I(l) &= k \cdot I_h(l) = k \cdot I_0 e^{-k l}
\end{aligned}
$${#eq-beers-law}

where $l_0$, $I_h(l)$ and $I(l)$ are the irradiances in PAR wavebands incident on the top of the canopy, on the horizontal surface at depth $l$, and on a leaf at depth $l$ accounting for additional extinction due to leaf angle, respectively. $k$ is the canopy extinction coefficient determined by canopy traits such as leaf angle and clumping. A more realistic light distribution model will consider other factors like leaf reflectance, transmission, and direct and diffuse components of irradiance which we will ignore here for simplicity [See @dePury1997; @Johnson2010]. With $I(l)$ known, we can apply a leaf photosynthesis model of our choice and integrate it over the total leaf area index ($L$) of the canopy to get canopy photosynthesis. We will use the rectangular hyperbola as an example. Combining @eq-beers-law with @eq-rectangular and ignoring respiration, (gross) leaf photosynthesis ($A(l)$) at canopy depth $l$ becomes:

$$
\begin{aligned}
A(l) = \frac{\alpha I(l) \cdot A_{\mathrm{max}}}{\alpha I(l) + A_{\mathrm{max}}} 
     = \frac{A_{\mathrm{max}} \cdot \alpha k I_0 e^{-k l}}{ A_{\mathrm{max}} + \alpha k I_0 e^{-k l}}
\end{aligned}
$${#eq-Acan-l}

Integrating this over the total canopy depth of LAI ($L$) gives the instantaneous canopy gross photosynthesis as shown in [@Goudriaan1986] and [@G&vL-1994]:

$$
    A_{g,\mathrm{can}} = \int_0^L A(l) \, dl = \frac{A_{\mathrm{max}}}{k} \cdot \ln{\frac{A_{\mathrm{max}} + \alpha k I_0}{A_{\mathrm{max}} + \alpha k I_0 e^{-k L}}}
$${#eq-Acan-sln}

Step by step derivation of this analytical solution can be found in [@Johnson1984]. Having an analytical solution is like this is very helpful for reducing errors from numerical integration as well as for computational efficiency. We will use this analytical solution to test the efficiency of different numerical integration methods as described in [@G&vL-1994].

::: {#nte-analytic-sln}
Think about the reasons why having an analytical solution can be beneficial for modeling canopy photosynthesis. Consider the fact that what we looked in this chapter is *instantaneous* photosynthesis while plant growth is a result of biomass accumulation over a period of time usually corresponding to the growing season.
:::

::: {#exr-Acan}
### Instantaneous canopy photosynthesis
Using `Cropbox`, let's evaluate three different systems for modeling **instantaneous canopy photosynthesis (**$A_{g,can}$) based on the Appendix (section A.2) of [@G&vL-1994] and also [@Goudriaan1986]:

1.  Build a system based on 1) analytical solution, 2) Gaussian 3 point integration method, and 3) Rectangular integration (Eulerian) method over total LAI ($L$) with an increment ($dl$) = 1.0 and 0.1 of LAI.

2.  Reproduce the results shown in Appendix (section A.2) of [@G&vL-1994] for the same condition.

3.  Plot *gross* canopy photosynthesis ($A_{g,can}$) to compare the three approaches in response to LAI ranging from 0 to 10.

4.  Plot *net* canopy photosynthesis ($A_{n,can}$) to compare these approaches in response to LAI ranging from 0 to 10. Assume that canopy respiration is 10% of $A_{\mathrm{max}}$ integrated over the whole canopy. Interpret the results and discuss their biological context.

:::{#tbl-Acan-params}
| Symbol | Value | Units | Description |
|:-|:-|:---|:-------|
| $l$ | \- | $\mathrm{m_{leaf}^2}\ \mathrm{m_{ground}^{-2}}$ | Cumulative leaf area index up to the position of a leaf inside the canopy |
| $I_h(l)$ | \- | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | $I$ incident on a horizontal surface at canopy depth $l$ |
| $I(l)$ | \- | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | $I$ incident on a leaf surface at canopy depth $l$ |
| $I_0$ | 575 | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Irradiance at the top of the canopy |
| $k$ | 0.8 | $\mathrm{m_{ground}^2}\ \mathrm{m_{leaf}^{-2}}$ | Light extinction coefficient inside the canopy |
| $L$ | \- | $\mathrm{m_{leaf}^2}\ \mathrm{m_{ground}^{-2}}$ | Total leaf area index of the canopy |
| $R_d$ | \- | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Canopy respiration rate |
| $\alpha$ | 0.0494 | $\mathrm{\mu mol_{CO_2}}\ \mathrm{\mu mol_{photon}^{-1}}$ | Apparent photochemical efficiency (*a.k.a.* quantum yield) |
| $A_{\mathrm{max}}$ | 22.73 | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Light saturated $A_g$ at ambient $\mathrm{[CO_2]}$ and standard temperature |
| $A(l)$ | \- | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Leaf gross $\mathrm{CO_2}$ assimilation rate ($A_g$) of a leaf inside the canopy at depth $l$ |

Variables and parameters of instantaneous canopy photosynthesis model
:::

In [ ]:
@system Irradiance begin
    I0: irradiance_top   => 575 ~ preserve(parameter, u"μmol/m^2/s") # equivalent to 125 W m-2 of PAR
    k:  extinction_coeff => 0.8 ~ preserve(parameter, u"m^2/m^2")

    Ih(I0, k; l) => I0 * exp(-k*l) ~ call(u"μmol/m^2/s")
    I(Ih, k; l)  => k * Ih(l)      ~ call(u"μmol/m^2/s")
end;

#### Breaking down the model into components: $A_{can}$ {.unnumbered}

`CanopyPhotosynthesis` consists of variables calculated by integrating over a depth of the canopy. Gross photosynthesis (`Ag`) is a `hold` variable implying an actual definition of this variable must be supplied by other systems (mix-ins) which will be later combined to form a complete system. Dark respiration (`Rd`) is calculated by the ratio parameter (`Rdp`) whose initial value is 0.1 assuming 10% of maximum gross photosynthesis (`Amax*L`) is used for canopy respiration (`Rd`).

$$R_d = 0.1 \cdot A_{\mathrm{max}} L$$


In [ ]:
c = @config Photosynthesis => (Amax = 22.73, α = 0.0494);

In [ ]:
@system CanopyPhotosynthesis(Photosynthesis) begin
    α:    photochemical_efficiency    => 0.0494 ~ preserve(parameter, u"μmol/μmol")
    Amax: maximum_photosynthetic_rate => 22.73  ~ preserve(parameter, u"μmol/m^2/s")

    L: leaf_area_index => 5 ~ preserve(parameter, u"m^2/m^2") # for comparison with Gourdriaan and van Laar (1994)

    Rdp:              dark_respiration_ratio => 0.1        ~ preserve(parameter)
    Rd(Rdp, Amax, L): dark_respiration       => Rdp*Amax*L ~ track(u"μmol/m^2/s")
end;

#### Analytical solution {.unnumbered}

Let's define an anlytical version of the model first. `CanopyPhotosynthesisA` is composed of other systems including `CanopyPhotosynthesis` and `Irradiance` defined above. These mixins provide actual definition of many variables declared to be `hold` here. For example, `α`, `Amax`, and `Rd` `Ag` is provided by this system while other variables are declared to be `hold` meaning provided by other mixins.

$$
A_{g,\mathrm{can}} = \int_0^L A(l) \, dl = \frac{A_{\mathrm{max}}}{k} \cdot \ln{\frac{A_{\mathrm{max}} + \alpha k I_0}{A_{\mathrm{max}} + \alpha k I_0 e^{-k L}}}
$$

In [ ]:
@system CanopyPhotosynthesisA(CanopyPhotosynthesis, Irradiance, Controller) begin
    Ag(α, Amax, I0, k, L): gross_photosynthesis => begin
        Amax/k * log((Amax + α*k*I0) / (Amax + α*k*I0*exp(-k*L)))
    end ~ track(u"μmol/m^2/s")
end;

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisA)

`CanopyPhotosynthesisN` is an intermediate mix-in providing a common ground for Gauss and Euler version of the model. Instead of directly computing `Ag` for the whole canopy, it makes uses a `call` variable `A` for calculating a photosynthesis rate at a certain layer represented by a value of leaf area index (`L`). `A` will be later integrated over a total range of leaf area index from 0 to `L` for computing `Ag`.


$$
A(l) = \frac{\alpha I(l) \cdot A_{\mathrm{max}}}{\alpha I(l) + A_{\mathrm{max}}}
$$


In [ ]:
@system CanopyPhotosynthesisN(CanopyPhotosynthesis, Irradiance, Controller) begin
    A(α, Amax, I; l) => begin
        Il = I(l)
        α*Il * Amax / (α*Il + Amax)
    end ~ call(u"μmol/m^2/s")
end;

#### Gaussian integration {.unnumbered}

`CanopyPhotosynthesisG` uses Gaussian integration method to compute `Ag` from `A` defined above. Let's use QuadGK for convenience.

In [ ]:
@system CanopyPhotosynthesisG(CanopyPhotosynthesisN, Controller) begin
    Ag(A, L): gross_photosynthesis => begin
        #Cropbox.QuadGK.quadgk(A, 0, L) |> first
        X, W = Cropbox.QuadGK.gauss(3, 0, L)
        sum(W .* A.(X))
    end ~ track(u"μmol/m^2/s")
end;

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisG)

#### Rectangular integration (Eulerian) {.unnumbered}

`CanopyPhotosynthesisE` uses Euler integration method to compute `Ag`. Overall structure is close to `CanopyPhotosynthesisG` except that it uses a parameter `dl` to control the size of integration step.

In [ ]:
@system CanopyPhotosynthesisE(CanopyPhotosynthesisN, Controller) begin
    dl => 1.0 ~ preserve(parameter)
    Ag(A, L, dl): gross_photosynthesis => begin
        sum([A(l) * dl for l in dl:dl:L])
    end ~ track(u"μmol/m^2/s")
end;

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisE);

Now it's time to make a plot of three models using the same option. We want to plot gross photosynthesis (`Ag`) against a range of leaf area index (`L`) from 0 to 15. `visualize()` function again conveniently provides a feature to draw a series of simulation lines from multiple systems.

Reproduce the results for three methods and compare them with the values from Gourdriaan and van Laar (1994) p. 181. Their analytical solution was 843.74 $\mathrm{\mu g}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ and Gaussian solution was 844.22 $\mathrm{\mu g}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ respectively. Note that 1 $\mathrm{\mu mol_{CO_2}}$ equals to 44 $\mathrm{\mu g_{CO_2}}$.

In [ ]:
co2_molar_mass = 44u"μg" / 1u"μmol";

In [ ]:
instance(CanopyPhotosynthesisA).Ag' * co2_molar_mass

In [ ]:
instance(CanopyPhotosynthesisG).Ag' * co2_molar_mass

In [ ]:
instance(CanopyPhotosynthesisE).Ag' * co2_molar_mass

Let's try a smaller integration step for the rectangular method. $dl$ is now decreased to 0.1 from 1.0 above.

In [ ]:
instance(CanopyPhotosynthesisE, config = :0 => :dl => 0.1).Ag' * co2_molar_mass

In [ ]:
models = [CanopyPhotosynthesisA, CanopyPhotosynthesisG, CanopyPhotosynthesisE]
xstep = :0 => :L => 0:0.1:10
names = ["Analytic", "Gauss", "Euler"]
kind = :scatter
ylim = (0, 30);

Note that the line of Gauss model (yellow) is barely visible since it's mostly overlapped with the line of analytic solution (blue).

In [ ]:
visualize(models, :L, :Ag; xstep, names, kind, ylim);

Plotting net photosynthesis (`An`) which takes out 10% of gross respiration (`Ag`) as dark respiration (`Rd`) can be easily done in a similar way.

In [ ]:
visualize(models, :L, :An; xstep, names, kind, ylim);

Note that Cropbox also implements a special kind of variable named `integrate` to cover the need of numerical integration as we explored in this notebook. Instead of relying on two separate variables, one `call` defining a function to be integrated and another `track` manually doing numerical integration, `integrate` can describe the same logic using a single variable. It takes a similar form as `call` putting the name of internal variable to be integrated (`l` in this case) after semi-colon (`;`) in the argument list. Then the range of integration is specified by `from` and `to` tags defined inside the parenthesis following the state name (`integrate`). As we want to integrate from 0 to `L` and the default value of `from` and `to` are 0, we only need to specify `to` as `L`. As a bonus, unit conversion would be automatically handled if any of these variables were assigned some units. `CanopyPhotosynthesisG2` implemented with `call` should behave in the exactly same way as `CanopyPhotosynthesisG` implemented earlier with `call` and `track`.

In [ ]:
@system CanopyPhotosynthesisG2(CanopyPhotosynthesis, Irradiance, Controller) begin
    Ag(α, Amax, I; l): gross_photosynthesis => begin
        Il = I(l)
        α*Il * Amax / (α*Il + Amax)
    end ~ integrate(to = L, u"μmol/m^2/s")
end;

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisG2);

In [ ]:
Cropbox.dependency(CanopyPhotosynthesisG2);

In [ ]:
instance(CanopyPhotosynthesisG2);

### Sensitivity of Parameters

We can test sensitivty of parameters as we did earlier.

In [ ]:
visualize(CanopyPhotosynthesisG2, :Amax, :An;
    xstep = :0 => :Amax => 1:50,
    kind = :line,
);

In [ ]:
visualize(CanopyPhotosynthesisG2, :α, :An;
    xstep = :0 => :α => 0:0.001:0.1,
    xlim = (0, 0.1),
    kind = :line,
);

In [ ]:
visualize(CanopyPhotosynthesisG2, :k, :An;
    xstep = :0 => :k => 0:0.01:1,
    kind = :line,
);

In [ ]:
visualize(CanopyPhotosynthesisG2, :L, :An;
    xstep = :0 => :L => 0:0.1:10,
    kind = :line,
);

#### Light response of leaf and canopy photosynthesis {.unnumbered}

In [ ]:
p = visualize(CanopyPhotosynthesisG2, :I0, :An;
    config = :0 => :L => 5,
    xstep = :0 => :I0 => 0:10:2000,
    kind = :line,
    name = "An,can (L=5)",
)
visualize!(p, PhotosynthesisRH, :I, :An;
    xstep = :0 => :I => 0:10:2000,
    kind = :line,
    name = "An,leaf",
)

:::

:::{#tip-cropbox-manipulate .callout-tip}
### `manipulate` method
We can make an interactive plot using `manipulate()` function in `Cropbox`.

In [ ]:
#| output: false
Cropbox.Interact.WebIO.setup(:ijulia);

In [ ]:
#| output: false
manipulate(CanopyPhotosynthesisG2, :I0, :An;
    parameters = :0 => :L => 0:0.1:10,
    xstep = :0 => :I0 => 0:10:2000,
    ylim = (-10, 30),
    kind = :line,
)

In [ ]:
#| output: false
manipulate(parameters = :0 => :L => 0:0.1:10) do c
    p = visualize(CanopyPhotosynthesisG2, :I0, :An;
        config = c,
        xstep = :0 => :I0 => 0:10:2000,
        ylim = (-10, 30),
        kind = :line,
        name = "An,can",
    )
    visualize!(p, PhotosynthesisRH, :I, :An;
        xstep = :0 => :I => 0:10:2000,
        kind = :line,
        name = "An,leaf",
    )
end

:::


## Homework Problems {#sec-hw-photosyn}

This homework assignment is an extension of @exr-Acan.

1.  Using the Gaussian 3 point integration method, experiment with different parameter values for physiological (e.g., $A_{\mathrm{max}}$, $\alpha$, $\theta$) and canopy (e.g., $k$, $L$, $I_0$) traits to evaluate their impacts on canopy productivity; that is, perform sensitivity analyses of the parameters within a reasonable range of their expected values. **(1)** Plot *net* canopy photosynthesis ($A_{n,can}$) in response to different values of the parameters and **(2)** Interpret and discuss the results. *Hint:* Use `xstep` and `group` for visualization.

2.  Here we are integrating leaf photosynthesis spatially over the leaf area index of a canopy in steady state. Hence the results represent 'instantaneous' canopy photosynthesis at one time point. This gives a snapshot but is not a dynamic model that runs over a day or throughout the growing season. Discuss how this steady-state snapshot may be incorporated into a growth function in the form of $dW/dt = rW$ we've looked at before for temporal integration where $r$ is the specific growth rate. How would you do that? No coding is necessary for this part. Just discuss.

::: {#tbl-photosynthesis-parms}
| Symbol | Value | Units | Description |
|:-|:-|:---|:-------|
| $V_{\mathrm{max}}$ | 100 | \[Product\]/time (e.g., μmol m^2^ s^−−1^) | Potential rate of biochemical reaction for product synthesis (ex, Rubisco capacity) |
| $K_m$ | 400 | \[Substrate\] (e.g., M, ppm, μmol mol^−1^) | Michaelis-Menten constant where $V = \frac{1}{2} V_{\mathrm{max}}$\| |
| $W$ | \- | g m^−2^, kg ha^−1^, t ha^−1^ | Plant or canopy biomass as dry matter |
| $\epsilon$ | 1.5, 2.3 | g MJ^−1^ | Typical radiation use efficiency for C3 and C4 species |
| $k$ | 0.8 | m^2^  m^−2^  | Light extinction coefficient inside the canopy. This value will vary with canopy properties typically ranging between 0.4 and 0.9. |
| $I_0$ | \- | W m^−2^, μmol m^2^ s^−−1^ | Irradiance at the top of the canopy in short-wave solar radiation (PAR+NIR wavebands). The same notation is sometimes used for photosynthetic photon flux density (PFD) |
| $L$ | \- | m^2^  m^−2^  | Total leaf area index (LAI) |
| $f_{\mathrm{solar}}$ | \- | \- | Fraction of the solar radiation intercepted by the canopy |
| $h$ | 0.5 | \- | Harvest index as the fraction of harvestable biomass over total biomass. 50% is a typical value for high yielding crops |
| $Y_h$ | \- | g m^−2^, kg ha^−1^, t ha^−1^ | Harvested crop yield |
| $A_g$ | \- | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Leaf gross assimilation |
| $A_n$ | \- | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Leaf net assimilation rate per leaf area per second |
| $\alpha$ | 0.05 | μmol  μmol^−1^  | Apparent photochemical efficiency (a.k.a., quantum yield) |
| $A_{\mathrm{max}}$ | 25.0 | μmol m^2^ s^−−1^ | Light saturated $A_g$ at ambient and standard temperatures (e.g., 25 °C). The values shown here are reasonable estimates for C3 crops like wheat, rose, and soybean. |
| $\theta$ | 0.7 | \- | Curvature factor determining the transition between light- limited and light-saturated (i.e., limited) photosynthesis. A value between 0.5 and 0.7 is typical for C3 leaves.\\ |
| $L$ | \- | m ^2^ m^−1^ ^2^ | Total leaf area index of the canopy |
| $l$ | \- | m ^2^ m^−1^ ^2^ | Cumulative leaf area index up to the position of a leaf inside the canopy |
| $I_0$ | 125, 575 | W m^−2^, μmol m^2^ s^−−1^ | Irradiance at the top of the canopy. The value here (125 W m^−2^) is for PAR only and is equivalent to 575 μmol m^2^ s^−−1^ of photosynthetic photon flux density (PFD) (4.6 photons/J of PAR in sun light) |
| $I_h(l)$ | \- | W m^−2^, μmol m^2^ s^−−1^ | $I$ incident on a horizontal surface at canopy depth $l$. |
| $I(l)$ | \- | W m^−2^, μmol m^2^ s^−−1^ | $I$ incident on a leaf surface at canopy depth $l$. This accounts for additional attenuation of light by the properties of the leaf itself. |
| $\alpha$ | 0.0494, 10 | μmol  μmol^−1^ , μg  J^−1^  | Apparent photochemical efficiency (a.k.a., quantum yield) as used in Appendix 2 of [@G&vL-1994] |
| $A(l)$ | \- | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Leaf gross assimilation rate ($A_g$) of a leaf inside the canopy at depth $l$. |
| $A_{g,\mathrm{can}}$ | \- | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Instantaneous canopy gross photosynthesis rate with a total leaf area index of $L$. |
| $A_{n,\mathrm{can}}$ | \- | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Instantaneous canopy net photosynthesis rate with a total leaf area index of $L$. |
| $R_d$ | $0.1 A_{\mathrm{max}}L$ | μmol m^2^ s^−−1^, μg m^2^ s^−−1^ | Canopy respiration rate |

Variables and parameters used in this section
:::